<h1><center>Anomaly Detection in IoT sensors using machine learning</center></h1>


<h4>About this Notebook</h4>
This notebook is created to detect anomaly in IoT sensors. After doing research, we found out that KNN algorithm is the most suitable to use in this program.


## Table of Contents

1. [Importing needed packages and training data](#0)<br>
2. [Pre-processing](#2)<br>
3. [Build the KNN model with k = 3](#4) <br>
4. [Save model to string using pickle](#6) <br> 
5. [Loading and preprocessing testing data](#8) <br> 
6. [Result](#10) <br> 
</div>
<hr>

## 1. Importing Needed packages and training data<a id="0"></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
matplotlib.style.use('ggplot')
%matplotlib inline

In [1]:
# The code was removed by Watson Studio for sharing.

,sourceID,sourceAddress,sourceType,sourceLocation,destinationServiceAddress,destinationServiceType,destinationLocation,accessedNodeAddress,accessedNodeType,operation,value,timestamp,normality
0,lightcontrol2,/agent2/lightcontrol2,/lightControler,BedroomParents,/agent2/lightcontrol2,/lightControler,BedroomParents,/agent2/lightcontrol2,/lightControler,registerService,none,1520031600000,normal
1,lightcontrol3,/agent3/lightcontrol3,/lightControler,Dinningroom,/agent3/lightcontrol3,/lightControler,Dinningroom,/agent3/lightcontrol3,/lightControler,registerService,none,1520031603269,normal
2,lightcontrol1,/agent1/lightcontrol1,/lightControler,BedroomChildren,/agent1/lightcontrol1,/lightControler,BedroomChildren,/agent1/lightcontrol1,/lightControler,registerService,none,1520031603279,normal
3,lightcontrol4,/agent4/lightcontrol4,/lightControler,Kitchen,/agent4/lightcontrol4,/lightControler,Kitchen,/agent4/lightcontrol4,/lightControler,registerService,none,1520031603290,normal
4,movement4,/agent4/movement4,/movementSensor,Kitchen,/agent4/movement4,/movementSensor,Kitchen,/agent4/movement4,/movementSensor,registerService,none,1520031603464,normal


In [2]:

body = client_aefa0132d2b14df5a0e73c7c8923d2af.get_object(Bucket='anomalydetectioniniotsensors-donotdelete-pr-uttlkb2u28atlz',Key='data.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df = pd.read_csv(body)
df.head()


,sourceID,sourceAddress,sourceType,sourceLocation,destinationServiceAddress,destinationServiceType,destinationLocation,accessedNodeAddress,accessedNodeType,operation,value,normality
0,lightcontrol2,/agent2/lightcontrol2,/lightControler,BedroomParents,/agent2/lightcontrol2,/lightControler,BedroomParents,/agent2/lightcontrol2,/lightControler,registerService,0.0,normal
1,lightcontrol3,/agent3/lightcontrol3,/lightControler,Dinningroom,/agent3/lightcontrol3,/lightControler,Dinningroom,/agent3/lightcontrol3,/lightControler,registerService,0.0,normal
2,lightcontrol1,/agent1/lightcontrol1,/lightControler,BedroomChildren,/agent1/lightcontrol1,/lightControler,BedroomChildren,/agent1/lightcontrol1,/lightControler,registerService,0.0,normal
3,lightcontrol4,/agent4/lightcontrol4,/lightControler,Kitchen,/agent4/lightcontrol4,/lightControler,Kitchen,/agent4/lightcontrol4,/lightControler,registerService,0.0,normal
4,movement4,/agent4/movement4,/movementSensor,Kitchen,/agent4/movement4,/movementSensor,Kitchen,/agent4/movement4,/movementSensor,registerService,0.0,normal


## 2. Pre-processing<a id="2"></a>

In [4]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
df["accessedNodeType_code"]= le.fit_transform(df["accessedNodeType"])
df["sourceID_code"]= le.fit_transform(df["sourceID"])
df["sourceAddress_code"]= le.fit_transform(df["sourceAddress"])
df["sourceType_code"]= le.fit_transform(df["sourceType"])
df["sourceLocation_code"]= le.fit_transform(df["sourceLocation"])
df["destinationServiceAddress_code"]= le.fit_transform(df["destinationServiceAddress"])
df["destinationServiceType_code"]= le.fit_transform(df["destinationServiceType"])
df["destinationLocation_code"]= le.fit_transform(df["destinationLocation"])
df["accessedNodeAddress_code"]= le.fit_transform(df["accessedNodeAddress"])
df["operation_code"]= le.fit_transform(df["operation"])
df["normality_code"]= le.fit_transform(df["normality"])

In [5]:
numeric_cols = ['timestamp','value']
categorical_cols = list(set(data.columns.values.tolist()) - set(numeric_cols))
df = df.drop(columns = categorical_cols)

In [6]:
df['value'] = df.value.astype(float)
df.dtypes

value                             float64
accessedNodeType_code               int64
sourceID_code                       int64
sourceAddress_code                  int64
sourceType_code                     int64
sourceLocation_code                 int64
destinationServiceAddress_code      int64
destinationServiceType_code         int64
destinationLocation_code            int64
accessedNodeAddress_code            int64
operation_code                      int64
normality_code                      int64
dtype: object

In [7]:
X = df.drop('normality_code', axis = 1 )
y = df.normality_code

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

In [9]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:5: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:6: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.


##  3. Build the KNN model with k = 3 <a id="4"></a>

In [10]:
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
k=3
knn = KNeighborsClassifier(n_neighbors = k).fit(X_train,y_train)

In [11]:
y_predict = knn.predict( X_test )

In [12]:
num_correct = ( ( y_predict == y_test.values ) == True ).sum()
print( "Success rate: " + str(  100 * ( num_correct / len( y_predict ) ) )  + "%" )

Success rate: 99.43287112485216%


## 4. Save model to string using pickle<a id="6"></a>

In [13]:
import pickle 
  
# Save the trained model as a pickle string.
saved_model = pickle.dumps(knn)

In [14]:
knn_from_pickle = pickle.loads(saved_model) 

## 5. Loading and preprocessing testing data<a id="8"></a>

In [15]:
body = client_aefa0132d2b14df5a0e73c7c8923d2af.get_object(Bucket='anomalydetectioniniotsensors-donotdelete-pr-uttlkb2u28atlz',Key='anomalous3.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

data_test = pd.read_csv(body)
data_test.head()


,sourceID,sourceAddress,sourceType,sourceLocation,destinationServiceAddress,destinationServiceType,destinationLocation,accessedNodeAddress,accessedNodeType,operation,value
0,washingmachine1,/agent6/washingmachine1,/washingService,Bathroom,/agent1/lightcontrol1,/lightControler,BedroomChildren,/agent1/lightcontrol1/lightOn,/derived/boolean,write,1.0000
1,washingmachine1,/agent6/washingmachine1,/washingService,Bathroom,/agent2/lightcontrol2,/lightControler,BedroomParents,/agent2/lightcontrol2/lightOn,/derived/boolean,write,1.0000
2,washingmachine1,/agent6/washingmachine1,/washingService,Bathroom,/agent2/tempin2,/sensorService,BedroomParents,/agent2/tempin2,/sensorService,read,19.9497
3,washingmachine1,/agent6/washingmachine1,/washingService,Bathroom,/agent3/tempin3,/sensorService,Dinningroom,/agent3/tempin3,/sensorService,read,19.9497
4,washingmachine1,/agent6/washingmachine1,/washingService,Bathroom,/agent4/tempin4,/sensorService,Kitchen,/agent4/tempin4,/sensorService,read,19.9497


In [16]:
le = preprocessing.LabelEncoder()
data_test["accessedNodeType_code"]= le.fit_transform(data_test["accessedNodeType"])
data_test["sourceID_code"]= le.fit_transform(data_test["sourceID"])
data_test["sourceAddress_code"]= le.fit_transform(data_test["sourceAddress"])
data_test["sourceType_code"]= le.fit_transform(data_test["sourceType"])
data_test["sourceLocation_code"]= le.fit_transform(data_test["sourceLocation"])
data_test["destinationServiceAddress_code"]= le.fit_transform(data_test["destinationServiceAddress"])
data_test["destinationServiceType_code"]= le.fit_transform(data_test["destinationServiceType"])
data_test["destinationLocation_code"]= le.fit_transform(data_test["destinationLocation"])
data_test["accessedNodeAddress_code"]= le.fit_transform(data_test["accessedNodeAddress"])
data_test["operation_code"]= le.fit_transform(data_test["operation"])


data_test['value'] = data_test.value.astype(float)

numeric_cols = ['timestamp','value','normality']
categorical_cols = list(set(data.columns.values.tolist()) - set(numeric_cols))
data_test = data_test.drop(columns = categorical_cols)

data_test = scaler.transform(data_test)

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:20: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.


## 6. Result <a id="10"></a>

In [17]:
y_predict_knn = knn_from_pickle.predict(data_test)

In [18]:
def check_anomaly(list1, val):
    # traverse in the list
    for x in list1:

        # compare with all the values
        # with val
        if x != val:
            return True
    return True

In [19]:
import collections
print(collections.Counter(y_predict_knn))
if check_anomaly(y_predict_knn, 7):
    print('Anomaly Detected')

Counter({7: 1155, 4: 264, 5: 128})
Anomaly Detected
